## Implementation of 
# Spectral-Spatial Hyperspectral Unmixing Using Multitask Learning
B. Palsson, J. R. Sveinsson and M. O. Ulfarsson, "Spectral-Spatial Hyperspectral Unmixing Using Multitask Learning," in IEEE Access, vol. 7, pp. 148861-148872, 2019, doi: 10.1109/ACCESS.2019.2944072.

### Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras import initializers, constraints, layers, activations, regularizers
from tensorflow.python.ops import math_ops
from tensorflow.python.keras import backend as K
from tensorflow.python.framework import tensor_shape
from unmixing import HSI, plotEndmembers,SAD
from unmixing import plotEndmembersAndGT, plotAbundancesSimple, load_HSI, PlotWhileTraining
from sklearn.feature_extraction.image import extract_patches_2d
from scipy import io as sio
import os
import numpy as np
from numpy.linalg import inv
import warnings
import matplotlib
import matplotlib.pyplot as plt
#%warnings.filterwarnings("ignore")
%matplotlib inline

### Use CPU

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Class SumToOne
Also performs regularizations l1 and OSP 
$$L_\text{OSP}+\rho_2\sum_i L_1(\bf{h_i})$$

In [ ]:
class SumToOne(layers.Layer):
    def __init__(self, params, **kwargs):
        super(SumToOne, self).__init__(**kwargs)
        self.num_outputs = params['num_endmembers']
        self.params = params
        
    def l1_regularization(self,x):
        l1 = tf.reduce_sum(tf.abs(x))
        return self.params['l1'] * l1
    
    def osp_regularization(self,x):
        return self.params['osp']*OSP(x,self.params['num_endmembers'])
        
    def call(self, x):
        self.add_loss(self.l1_regularization(x))
        x = tf.nn.softmax(self.params['scale'] * x)
        return x

## Class Autoencoder
Wrapper class for the autoencoder model and associcated utility functions

In [ ]:
class Autoencoder(object):
    def __init__(self, params):
        self.data = None
        self.params = params
        self.model = self.create_model()
        self.model.compile(optimizer=self.params["optimizer"], loss=self.params["loss"])
    
        
    def create_model(self):
        input_ = []
        output = []
        #init = initializers.glorot_normal()
        init = initializers.RandomNormal(0.01, 0.1)
        init2 = initializers.RandomNormal(0.01, 0.1)
        for i in range(n_inputs):
            input_.append(layers.Input(shape=(self.params['n_bands'],), name='input' + str(i)))
        concatenated = layers.concatenate([input_[i] for i in range(n_inputs)])
        concatenated = layers.GaussianNoise(0.05)(concatenated)
        
        dense = layers.Dense(units=(self.params['n_bands'] * self.params['n_inputs']) // 4,
                      name='dense1',
                      activation=self.params['activation'],
                      use_bias=False,
                      kernel_initializer=init)(concatenated)
        dense = layers.Dropout(0.5)(dense)
        dense = layers.BatchNormalization()(dense)
        endmembers = layers.Dense(units=self.params['n_bands'],
                           activation='linear',
                           name='endmembers',
                           use_bias=False,
                           kernel_constraint=constraints.non_neg(),
                           kernel_initializer=init)
        for i in range(n_inputs):
            abund = layers.Dense(units=self.params['num_endmembers'],
                          name='dense_3' + str(i),
                          activation=self.params['activation'],
                          use_bias=False,
                          kernel_initializer=init)(dense)
            abund = layers.BatchNormalization()(abund)
            abund = layers.Dropout(0.1)(abund)
            abund = SumToOne(self.params, name='abundances' + str(i))(abund)
            output.append(endmembers(abund))

        return tf.keras.Model(inputs=[input_[i] for i in range(n_inputs)], outputs=[output[i] for i in range(n_inputs)])
    
    def fit(self,data):
        self.data = data
        num_inputs = self.params['n_inputs']
        return self.model.fit(
            [self.data[:, i, :] for i in range(num_inputs)],
            [self.data[:, i, :] for i in range(num_inputs)],
            batch_size=self.params["batch_size"],
            epochs=self.params["epochs"],
            verbose=1
        )
    

    def get_endmembers(self):
        return self.model.layers[len(self.model.layers) - 1].get_weights()[0]

    def get_abundances(self):
        intermediate_layer_model = tf.keras.Model(inputs=self.model.input,
                                         outputs=[self.model.get_layer('abundances' + str(i)).output for i in
                                                  range(self.params['n_inputs'])])
        abundances = np.mean(intermediate_layer_model.predict([self.params['data'].array() for i in range(self.params['n_inputs'])]),
                             axis=0)
        abundances = np.reshape(abundances,[self.params['data'].cols,self.params['data'].rows,self.params['num_endmembers']])
        return abundances
    

## Method make_patches
Makes $n\times n$ patches for MTL unmixer

In [ ]:
def make_patches(hsi, n, num_patches):
    patch_size = n
    data = extract_patches_2d(hsi.image, (n, n), num_patches)
    s = data.shape
    data = data.reshape(s[0], n * n, hsi.bands)
    return data

## Load Data and set Hyperparameters

In [ ]:
#Dictonary of aliases for datasets. The first string is the key and second is value (name of matfile without .mat suffix)
#Useful when looping over datasets
datasetnames = {
        "Urban": "Urban4",
}

dataset = "Urban"

hsi = load_HSI(
    "./Datasets/" +datasetnames[dataset] + ".mat"
)

# Hyperparameters
num_endmembers = 4
n_inputs = 4
num_patches = 2000
batch_size = 15
learning_rate = 0.001
epochs = 60
loss = SAD
activation = layers.LeakyReLU(0.2)
l1 = 0

# Hyperparameter dictionary
params = {
    'n_inputs':n_inputs,
    "activation": activation,
    "num_endmembers": num_endmembers,
    "batch_size": batch_size,
    "num_patches": num_patches,
    "data": hsi,
    "epochs": epochs,
    "n_bands": hsi.bands,
    "GT": hsi.gt,
    "lr": learning_rate,
    "optimizer": tf.optimizers.RMSprop(learning_rate=learning_rate,decay=0.000),
    "loss": loss,
    "scale": 3,
    "l1": l1,
}

training_data = make_patches(hsi,params['n_inputs'],params['num_patches'])


## Train Autoencoder

## Run experiment 

In [ ]:
num_runs = 25
results_folder = './Results'
method_name = 'MTAEU'

for dataset in ['Urban']:
    save_folder = results_folder+'/'+method_name+'/'+dataset
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    hsi = load_HSI("./Datasets/" + datasetnames[dataset] + ".mat")

    for run in range(1,num_runs+1):
        print('run nr: '+str(run)+'\n')
        params = {
        'n_inputs':n_inputs,
        "activation": activation,
        "num_endmembers": num_endmembers,
        "batch_size": batch_size,
        "num_spectra": num_spectra,
        "data": hsi,
        "epochs": epochs,
        "n_bands": hsi.bands,
        "GT": hsi.gt,
        "lr": learning_rate,
        "optimizer": tf.optimizers.RMSprop(learning_rate=learning_rate,decay=0.0001),
        "loss": loss,
        "scale": 3,
        "l1": l1,
        }
        training_data = make_patches(hsi,params['n_inputs'],params['num_spectra'])
        save_folder = results_folder+'/'+method_name+'/'+dataset
        save_name = dataset+'_run'+str(run)+'.mat'
        save_path = save_folder+'/'+save_name
        autoencoder = Autoencoder(params)
        autoencoder.fit(training_data)
        endmembers = autoencoder.get_endmembers()
        abundances = autoencoder.get_abundances()
        plotEndmembersAndGT(endmembers, hsi.gt)
        plotAbundancesSimple(abundances,'abund.png')
        sio.savemat(save_path,{'M':endmembers,'A':abundances})
        del autoencoder
    